<h1 align="center"><font size="7" >Web scraping</font></h1>

# Introduction

In this simple project, I propose a classic way to gather the data from the web. I use a free web site of vehicle advertisements in France [**La centrale**](https://www.lacentrale.fr/) to gather the annonyme data of vehicles.
\
\
**Important remark**
\
When I made this project, the weeb site "La Central" didn't verify proptly if there is a robot or not. In the future the website can introduce a robot verification and the below code will not woork correctely. 
\
Actually the website ask a verification as you can see below, but the weeb scraping work very well anyway, so I think that this verification isn't efficace.


<p style="text-align:center">
    <a>
    <img src="img/img2.png"  >
    </a>
</p>

# Calling the necessary libraries
**Reminder** : you can easily install a new library in Jupyter by the command **!pip install name_of_labrary**. \
Example: "!pip install bs4". for more information please please see the [link](https://pypi.org/project/bs4/) of bs4

In [1]:
################## Importing libraries #############################
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import warnings
import datetime
import pandas as pd
import numpy as np

**Initialisation**

In [2]:
warnings.filterwarnings('ignore')
df=pd.DataFrame(columns=['page','href','model','version','price','dep','prof','warranty','year','km'])
Nb=0

# The loop of each page of the website

#### the main loop "for i in range (1,501)"

The below part is the main part of the project. The main loop change the page index each time, so it open the page 1 and 2 and soone. We can change manually the page number "1" in the link 'https://www.lacentrale.fr/listing?page=1' by the number 2 or other number.\
I remark manually that the maximum number of pages is 500, because of that the for loop change the index "i" from 1 to 500 (see below).
\
\
In this loop, the programme open the html file by **urlopen** and extract data from HTML by using [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/).


#### the second loop "for d in divs":
We can remark manually that each page shows 15 or 16 vehicles. This loop extract the data from each vehicle in the same page. especially the link of each vehicle by the command href='https...
We can find data of each vehicle page in soup2. and the rest of code will read the needed data by using **find_all** command.
\
\
To know which data you must read and how you can click on the **Inspection** by the right mouse click in the needed area, see the screenshot below: 

<img src="img/img1.png" >

In [3]:
for i in range (1,501): ########## Looping for each weebsite pages, the weebsite show 500 pages as maximum, 16 vehicles by page #########
    try:
        ################### Opening each page
        url=r'https://www.lacentrale.fr/listing?page='+str(i)
        userAgent = "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.86 Safari/537.36"
        req = Request(url, None, {'User-Agent': userAgent})
        html = urlopen(req).read()
        soup= BeautifulSoup(html,"html.parser")
        divs = soup.find_all("div", {"class": "searchCard__rightContainer"})
    except:
        break  ############ Break if the page didn't open  ######################
    for d in divs:
        try:
            ################# gatherig the data from each page  ###################
            href='https://www.lacentrale.fr'+d.parent.parent['href']
            model=d.h3.find_all('span', {"class": "searchCard__makeModel"})[0].text
            version=d.h3.find_all('span', {"class": "searchCard__version"})[0].text
            price=d.div.div.select('div[class^="searchCard__fieldPrice"]')[0].find_all('span')[-1].text
            dep=d.select('div[class^="searchCard__customerLocalisation"]')[0].select('div[class^="searchCard__dptCont"]')[0].text
            prof=d.select('div[class^="searchCard__customerLocalisation"]')[0].select('div[class^="cbm-txt--default searchCard__customer"]')[0].text
            warranty=d.select('div[class^="searchCard__warrantyYearMileage"]')[0].select('div[class^="searchCard__warranty"]')[0].span.text
            year=d.select('div[class^="searchCard__warrantyYearMileage"]')[0].select('div[class^="searchCard__year"]')[0].span.text
            km=d.select('div[class^="searchCard__warrantyYearMileage"]')[0].select('div[class^="searchCard__mileage"]')[0].span.text
            dic={'page':i,'href':href,'model':model,'version':version,'price':price,'dep':dep,\
                            'prof':prof,'warranty':warranty,'year':year,'km':km}
            ######################################## Open the link of each vehicle ##########################
            req2 = Request(href, None, {'User-Agent': userAgent})
            html2 = urlopen(req2).read()
            soup2= BeautifulSoup(html2,"html.parser")
            div2= soup2.find_all("div", {"class": "cbm-moduleInfos__informationList cbm-moduleInfos__information_column_break"})
            ul=div2[0].find_all('ul')
            #################### gathering data from link of each vehicle ######################
            for u in ul:
                li=u.find_all('li')
                for l in li:
                    span=l.find_all('span')
                    dic[span[0].text]=span[1].text
                    
            myli = soup2.find_all("li", {"class": "list-item"})
            for each in myli:
                strr=each.span.get_text()
                if 'Volume du coffre :' in strr or 'Longueur :'in strr :
                    strr=strr.split(':')
                    dic[strr[0]]=strr[1]
                    
            for myspan in soup2.find_all("span", {"class": "headerSection-extraContent"}):
               dic['Ref_annonce']=myspan.get_text().split('Réf. annonce : ')[1]

            for a in soup2.find_all("a", {"class": "link button-theme4"}):
                if 'équipements & options' in a.get_text():
                    dic['Nb_option']=a.get_text().split('équipements & options')[0]
            
            publication=soup2.find_all("div", {"class": "cbm-toolboxButtons"})[0].span.text.split('\n')[1]
            dic['publication']=publication
            
            ######################## Add data to the dataframe 
            df = df.append(dic,ignore_index=True)
            Nb=Nb+1
        except:pass

    print("Page",i,"/ 500 Nb vehicles",Nb)


Page 1 / 500 Nb vehicles 14
Page 2 / 500 Nb vehicles 30


\
\
This programme is not very fast, it took me one hour to gather the data of 7500 vehicles with my computer.
\
\
**display the dataframe data**

In [4]:
df

,page,href,model,version,price,dep,prof,warranty,year,km,...,Autonomie constructeur? :,Capacité batterie :,Voltage batterie :,Intensité batterie :,Conso. batterie :,Prix inclut la batterie :,Garantie constructeur? :,Vérifié & Garanti? :,Prime à la conversion? :,Provenance :
0,1,https://www.lacentrale.fr/auto-occasion-annonc...,JEEP WRANGLER 4,IV UNLIMITED 2.2 MJET 200 SAHARA AUTO,70 620 €,66,Professionnel,Garantie 12 mois,2019,23 956 km,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,https://www.lacentrale.fr/utilitaire-occasion-...,FORD TRANSIT 4,IV P 350 L4 TREND 2.0 L TDCI 130,33 200 €,24,Professionnel,Garantie 6 mois,2018,56 736 km,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,https://www.lacentrale.fr/auto-occasion-annonc...,AUDI E-TRON GT,93KWH GT QUATTRO RS S EXTENDED,181 330 €,92,Professionnel,Garantie 24 mois,2020,12 636 km,...,455 Km,93.4 kWh,800 V,40 A,19 kWh/100km,oui,en cours,NaN,NaN,NaN
3,1,https://www.lacentrale.fr/auto-occasion-annonc...,OPEL GRANDLAND X,1.2 ECOTEC TURBO 130 INNOVATION,21 380 €,27,Professionnel,Garantie 12 mois,2016,59 854 km,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SPOTICAR,NaN,NaN
4,1,https://www.lacentrale.fr/auto-occasion-annonc...,VOLKSWAGEN GOLF SPORTSVAN,(2) 1.6 TDI 115 BLUEMOTION TECHNOLOGY CONNECT ...,23 420 €,55,Professionnel,Garantie 12 mois,2018,82 530 km,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,https://www.lacentrale.fr/auto-occasion-annonc...,MINI MINI 3 5P,III 1.5 COOPER D 116 5P,11 320 €,01,Professionnel,Garantie 3 mois,2017,156 600 km,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1,https://www.lacentrale.fr/auto-occasion-annonc...,VOLKSWAGEN POLO 6 GTI,VI 2.0 TSI 200 GTI DSG6,30 480 €,28,Professionnel,Garantie 12 mois,2019,44 650 km,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1,https://www.lacentrale.fr/auto-occasion-annonc...,SEAT ARONA,1.0 TGI 90 S/S XCELLENCE BV6,20 940 €,69,Professionnel,Garantie 12 mois,2019,35 217 km,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN
8,1,https://www.lacentrale.fr/utilitaire-occasion-...,CITROEN C4 CACTUS BUSINESS,(2) 1.5 BLUEHDI 100 FEEL NAV,14 250 €,76,Professionnel,Garantie 8 mois,2018,101 947 km,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SPOTICAR,NaN,NaN
9,1,https://www.lacentrale.fr/auto-occasion-annonc...,DS DS 7 CROSSBACK,1.5 BLUEHDI 130 7CV GRAND CHIC AUTOMATIQUE,47 490 €,92,Professionnel,Garantie 12 mois,2023,17 731 km,...,NaN,NaN,NaN,NaN,NaN,NaN,en cours,SPOTICAR,NaN,NaN


**Saving the DataFrame as a csv file**

In [5]:
############### Save the finale file  #########################################"
file_name='lecentral_data'+datetime.datetime.now().strftime("%Y_%m_%d__%H_%M_%S")+'.csv'
df.to_csv(file_name,sep=';')

# Conclusion
This is a sample way to gother data from the web. 
as you can remark the website [**La centrale**](https://www.lacentrale.fr/) alows only 500 pages, 
each page has 15 or 16 vehicles, so we can gother a total around 7500 vehicles only. 
\
\
But the website has more than that, as we can see the example below, the website has more than 264 000 vehicles. A way to gather the data from all vehicle, is to complete the code above with a searching by batch, each time we can define automatically the maximum and minimum price to have around 7500 vehicles. We can integrate this in a loop and inside it we can loop the 500 pages of each price batch. This idea will be explained in detail in a incomping project.

<img src="img/img3.png" />

<p>
    <img src="img/img3.png" width="220" height="240" >
</p>

<img src="img/img1.png"  >